In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from IPython.display import clear_output
!pip3 install -U lazypredict
!pip3 install -U pandas #Upgrading pandas

#
clear_output()

In [ ]:
import pandas
pandas.__version__

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
# 원핫인코딩할때 썼던 거(label encoder)랑 정확도 측정때 사용했던 것들(accuracy_score)? k겹 교차검증(kFold)에 test데이터 스플릿함수 import(자세힌 기억이...)

from lightgbm import LGBMClassifier # LGBMClassifier가 지금 전체적으로 오류남.
import lazypredict
# Lazy predict - 튜닝 없이 Basic 모델을 이용했을 때, 어떤 모델을 이용하면 좋은 결과를 얻을 수 있는지 확인할 수 있다. 단, 튜닝이 없기에 맹신은 금물.

from lazypredict.Supervised import LazyClassifier # 얘도 마찬가지로 오류남.

import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
submission = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")

RANDOM_STATE = 12 
FOLDS = 5
STRATEGY = 'median'

In [ ]:
train.head()

In [ ]:
print(f'\033[94mNumber of rows in train data: {train.shape[0]}')
print(f'\033[94mNumber of columns in train data: {train.shape[1]}')
print(f'\033[94mNumber of values in train data: {train.count().sum()}')
print(f'\033[94mNumber missing values in train data: {sum(train.isna().sum())}')
#format해서 출력하는 형태. 그냥 shape만 써도 충분하긴 할거 같음.

In [ ]:
print(f'\033[94m')
print(train.isna().sum().sort_values(ascending = False))
#결측값 확인

In [ ]:
train.describe()

In [ ]:
print(f'\033[94mNumber of rows in test data: {test.shape[0]}')
print(f'\033[94mNumber of columns in test data: {test.shape[1]}')
print(f'\033[94mNumber of values in train data: {test.count().sum()}')
print(f'\033[94mNo of rows with missing values  in test data: {sum(test.isna().sum())}')

In [ ]:
print(f'\033[94m')
print((test.isna().sum().sort_values(ascending = False)))

In [ ]:
test.describe()

In [ ]:
submission.head()

In [ ]:
train.drop(["PassengerId"] , axis = 1 , inplace = True)
test.drop(["PassengerId"] , axis = 1 , inplace = True)
TARGET = 'Transported'
FEATURES = [col for col in train.columns if col != TARGET]
RANDOM_STATE = 12 

# passenger id 제거, target은 transported 하지 못한것(False)에 맞추어 features 걸러냄.

In [ ]:
# NULL 값들이 데이터에서 어떻게 분포해 있는지를 확인하는 코드.

test_null = pd.DataFrame(test.isna().sum())
test_null = test_null.sort_values(by = 0 ,ascending = False)
# test data의 null값들을 뽑아서 값을 정렬한다.
train_null = pd.DataFrame(train.isna().sum())
train_null = train_null.sort_values(by = 0 ,ascending = False)[:-1]
# train data의 null값들을 뽑아서 값을 정렬한다.

# subplots를 활용해서 그래프 2개를 나란히 출력할수 있게 한다.
fig = make_subplots(rows=1, 
                    cols=2,
                    column_titles = ["Train Data", "Test Data"] ,
                    x_title="Missing Values")

# 각각 그래프 추가
fig.add_trace(go.Bar(x=train_null[0],
                     y=train_null.index,
                     orientation="h",
                    marker=dict(color=[n for n in range(12)], 
                                line_color='rgb(0,0,0)' , 
                                line_width = 2,
                                coloraxis="coloraxis")),
              1, 1)

# 각각 그래프 추가
fig.add_trace(go.Bar(x=test_null[0], 
                     y=test_null.index,
                     orientation="h",
                    marker=dict(color=[n for n in range(12)], 
                                line_color='rgb(0,0,0)', 
                                line_width = 2,
                                coloraxis="coloraxis")),
              1, 2)

fig.update_layout(showlegend=False, title_text="Column wise Null Value Distribution", title_x=0.5)

In [ ]:
# 앞의 과정과 동일하게 진행하되, 이번엔 column이 아닌 row값 기준으로 분포를 구한다.
missing_train_row = train.isna().sum(axis=1)
missing_train_row = pd.DataFrame(missing_train_row.value_counts()/train.shape[0]).reset_index()
missing_test_row = test.isna().sum(axis=1)
missing_test_row = pd.DataFrame(missing_test_row.value_counts()/test.shape[0]).reset_index()
missing_train_row.columns = ['no', 'count']
missing_test_row.columns = ['no', 'count']
missing_train_row["count"] = missing_train_row["count"]*100
missing_test_row["count"] = missing_test_row["count"]*100


fig = make_subplots(rows=1, 
                    cols=2,
                    column_titles = ["Train Data", "Test Data"] ,
                    x_title="Missing Values",)

fig.add_trace(go.Bar(x=missing_train_row["no"], 
                     y=missing_train_row["count"]  ,
                    marker=dict(color=[n for n in range(4)], 
                                line_color='rgb(0,0,0)' ,
                                line_width = 3
                                ,coloraxis="coloraxis")),
              1, 1)
fig.add_trace(go.Bar(x= missing_test_row["no"], 
                     y=missing_test_row["count"],
                    marker=dict(color=[n for n in range(4)], 
                                line_color='rgb(0,0,0)',
                                line_width = 3 ,
                                coloraxis="coloraxis")),
              1, 2)
fig.update_layout(showlegend=False, title_text="Row wise Null Value Distribution", title_x=0.5)

In [ ]:
df = pd.concat([train[FEATURES], test[FEATURES]], axis=0)
text_features = ["Cabin", "Name"]

 # nunique = unique한 갯수를 나타내는 함수 -> 값이 어느정도 이상이면 continuous, 아니면 categorical data
cat_features = [col for col in FEATURES if df[col].nunique() < 25 and col not in text_features ]
cont_features = [col for col in FEATURES if df[col].nunique() >= 25 and col not in text_features ]

del df
print(f'\033[94mTotal number of features: {len(FEATURES)}')
print(f'\033[94mNumber of categorical features: {len(cat_features)}')
print(f'\033[94mNumber of continuos features: {len(cont_features)}')
print(f'\033[94mNumber of text features: {len(text_features)}')

labels=['Categorical', 'Continuos', "Text"]
values= [len(cat_features), len(cont_features), len(text_features)]
colors = ['#DE3163', '#58D68D']

fig = go.Figure(data=[go.Pie(
    labels=labels, 
    values=values, pull=[0.1, 0, 0 ],
    marker=dict(colors=colors, 
                line=dict(color='#000000', 
                          width=2))
)])
fig.show()

 # 데이터의 종류(분포)를 확인

In [ ]:
train.head()

In [ ]:
train_age = train.copy()
test_age = test.copy()

# train데이터와 test데이터를 따로 라벨링하여 구분(합쳤을때 구분 가능하도록)
train_age["type"] = "Train"
test_age["type"] = "Test"

# train데이터와 test데이터를 합침
ageDf = pd.concat([train_age, test_age])

# 나이의 분포를 히스토그램으로 나타내는 코드. 
fig = px.histogram(data_frame = ageDf, 
                   x="Age",
                   color= "type",
                   color_discrete_sequence =  ['#58D68D','#DE3163'],
                   marginal="box",
                   nbins= 100,
                    template="plotly_white"
                )
fig.update_layout(title = "Distribution of Age" , title_x = 0.5)
fig.show()

In [ ]:
# singleimputer 를 이용해서 결측치를 채워주는 코드 - dropna와 다르게 평균값, 중앙값, 최빈값, 정해진 값 결측치를 채울 값을 정하는 방법이 정해져 있고,
# 그 이미 정해진 strategy들 가운데에 하나를 골라 그 방법으로 결측치를 간단하게, 자동으로 채워줌.
imputer_cols = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck" ,"RoomService"]
imputer = SimpleImputer(strategy=STRATEGY )
imputer.fit(train[imputer_cols])
train[imputer_cols] = imputer.transform(train[imputer_cols])
test[imputer_cols] = imputer.transform(test[imputer_cols])

# train과 test데이터에서 안들어간 값들은 Z로 채워넣음(임시)
train["HomePlanet"].fillna('Z', inplace=True)
test["HomePlanet"].fillna('Z', inplace=True)

In [ ]:
# 문자로 된 데이터들로 이루어진 행들만 따로 모으기
label_cols = ["HomePlanet", "CryoSleep","Cabin", "Destination" ,"VIP"]

# 라벨 인코딩(문자열 = 컴퓨터가 이해할 수 있도록 범주화) 해주는 코드
def label_encoder(train,test,columns):
    for col in columns:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)
        train[col] = LabelEncoder().fit_transform(train[col])
        test[col] =  LabelEncoder().fit_transform(test[col])
    return train, test

# 라벨 인코딩 진행하기
train ,test = label_encoder(train,test ,label_cols)

In [ ]:
# Name과 Cabin은 예측값에 영향을 주지 않으므로 drop을 이용해 제거
train.drop(["Name" ,"Cabin"] , axis = 1 ,inplace = True)
test.drop(["Name" ,"Cabin"] , axis = 1 ,inplace = True)

# X, y(train데이터와 그 데이터에 대한 정답) 분리
X = train.drop(TARGET , axis =1)
y = train[TARGET]

# 그 중에서 train에 사용할 데이터와 test에 사용할 데이터로 또다시 분리
X_train , X_test , y_train , y_test = train_test_split(X , 
                                                       y, 
                                                       random_state = 12 ,
                                                       test_size =0.33)

In [ ]:
# 여러 모델들 중에서 가장 이 데이터를 예측하는데 적합한 모델을 찾기 위해
# lazy predict를 수행하는 코드
clf = LazyClassifier(verbose=0,
                     ignore_warnings=True,
                     custom_metric=None,
                     predictions=False,
                     random_state=12,
                     classifiers='all')

models, predictions = clf.fit(X_train , X_test , y_train , y_test)
clear_output()

In [ ]:
# LGBM이 가장 정확도가 높은? 코드이므로 LGBM을 사용, 사용을 위해 lgb의 파라미터 설정.
lgb_params = {
    'objective' : 'binary',
    'n_estimators' :50,
    'learning_rate' : 0.08
}

lgb_predictions = 0
lgb_scores = []
lgb_fimp = []

# 맨 마지막 열 제외하고 모든 데이터의 행들을 뽑아냄.
LGBM_FEATURES = list(train.columns)[:-1]

# K겹 교차검증과 LGBM을 동시에 수행
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)
for fold, (train_idx, valid_idx) in enumerate(skf.split(train[LGBM_FEATURES], train[TARGET])):
    print(f'\033[94m')
    print(10*"=", f"Fold={fold+1}", 10*"=")
    start_time = time.time()
    
    X_train, X_valid = train.iloc[train_idx][LGBM_FEATURES], train.iloc[valid_idx][LGBM_FEATURES]
    y_train , y_valid = train[TARGET].iloc[train_idx] , train[TARGET].iloc[valid_idx]
    
    model = LGBMClassifier(**lgb_params)
    model.fit(X_train, y_train,verbose=0)
    
    preds_valid = model.predict(X_valid)
    acc = accuracy_score(y_valid,  preds_valid)
    lgb_scores.append(acc)
    run_time = time.time() - start_time
    
    fim = pd.DataFrame(index=LGBM_FEATURES,
                 data=model.feature_importances_,
                 columns=[f'{fold}_importance'])
    lgb_fimp.append(fim)
    
    print(f"Fold={fold+1}, Accuracy score: {acc:.2f}%, Run Time: {run_time:.2f}s")
    test_preds = model.predict(test[LGBM_FEATURES]) 
    lgb_predictions += test_preds/FOLDS
    
print("")

# 각 시행 회차(K겹 교차검증)의 정확도를 출력
print("Mean Accuracy :", np.mean(lgb_scores))

In [ ]:
# submission파일로 제출하는 코드
submission[TARGET] = lgb_predictions.astype("bool")
submission.to_csv("submission.csv",index=False)
submission.head()